In [1]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# 資料路徑
DATASET_PATH  = 'data'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 20

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)



Found 3000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Class #0 = cats
Class #1 = dogs


C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 41s 0us/step


W0813 08:09:45.418452 141284 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

Epoch 1/20
375/375 [==============================] - 4334s 12s/step - loss: 0.8410 - acc: 0.7443 - val_loss: 0.1681 - val_acc: 0.9420
Epoch 2/20
375/375 [==============================] - 4336s 12s/step - loss: 0.4222 - acc: 0.8773 - val_loss: 0.1229 - val_acc: 0.9620
Epoch 3/20
375/375 [==============================] - 4329s 12s/step - loss: 0.3541 - acc: 0.9020 - val_loss: 0.0999 - val_acc: 0.9730
Epoch 4/20
375/375 [==============================] - 4319s 12s/step - loss: 0.2790 - acc: 0.9163 - val_loss: 0.0931 - val_acc: 0.9730
Epoch 5/20
375/375 [==============================] - 4307s 11s/step - loss: 0.2829 - acc: 0.9257 - val_loss: 0.0876 - val_acc: 0.9790
Epoch 6/20
375/375 [==============================] - 4311s 11s/step - loss: 0.2108 - acc: 0.9373 - val_loss: 0.0859 - val_acc: 0.9760
Epoch 7/20
375/375 [==============================] - 4314s 12s/step - loss: 0.1868 - acc: 0.9510 - val_loss: 0.0767 - val_acc: 0.9770
Epoch 8/20
375/375 [==============================] - 4

In [23]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import sys
import numpy as np
import pandas as pd

# 從參數讀取圖檔路徑

import os

path = 'data\\test'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files.append(os.path.join(r, file))
print (files)

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = ['cats', 'dogs']
cols=['ID', 'Predicted']
df=pd.DataFrame(columns=cols)
# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f.replace('data\\test\\','').replace('.jpg',''))
    print('    {:.3f}  {}'.format(pred[0], cls_list[0]))
    df=df.append({'ID':f.replace('data\\test\\','').replace('.jpg',''), 'Predicted':'{:.12f}'.format(pred[0])},ignore_index=True)

df

['data\\test\\000.jpg', 'data\\test\\001.jpg', 'data\\test\\002.jpg', 'data\\test\\003.jpg', 'data\\test\\004.jpg', 'data\\test\\005.jpg', 'data\\test\\006.jpg', 'data\\test\\007.jpg', 'data\\test\\008.jpg', 'data\\test\\009.jpg', 'data\\test\\010.jpg', 'data\\test\\011.jpg', 'data\\test\\012.jpg', 'data\\test\\013.jpg', 'data\\test\\014.jpg', 'data\\test\\015.jpg', 'data\\test\\016.jpg', 'data\\test\\017.jpg', 'data\\test\\018.jpg', 'data\\test\\019.jpg', 'data\\test\\020.jpg', 'data\\test\\021.jpg', 'data\\test\\022.jpg', 'data\\test\\023.jpg', 'data\\test\\024.jpg', 'data\\test\\025.jpg', 'data\\test\\026.jpg', 'data\\test\\027.jpg', 'data\\test\\028.jpg', 'data\\test\\029.jpg', 'data\\test\\030.jpg', 'data\\test\\031.jpg', 'data\\test\\032.jpg', 'data\\test\\033.jpg', 'data\\test\\034.jpg', 'data\\test\\035.jpg', 'data\\test\\036.jpg', 'data\\test\\037.jpg', 'data\\test\\038.jpg', 'data\\test\\039.jpg', 'data\\test\\040.jpg', 'data\\test\\041.jpg', 'data\\test\\042.jpg', 'data\\tes

000
    1.000  cats
001
    0.000  cats
002
    1.000  cats
003
    0.000  cats
004
    1.000  cats
005
    1.000  cats
006
    0.000  cats
007
    1.000  cats
008
    1.000  cats
009
    1.000  cats
010
    0.000  cats
011
    1.000  cats
012
    0.000  cats
013
    1.000  cats
014
    1.000  cats
015
    0.000  cats
016
    0.000  cats
017
    1.000  cats
018
    0.000  cats
019
    1.000  cats
020
    1.000  cats
021
    1.000  cats
022
    1.000  cats
023
    1.000  cats
024
    1.000  cats
025
    0.000  cats
026
    0.000  cats
027
    0.000  cats
028
    0.000  cats
029
    0.000  cats
030
    0.000  cats
031
    0.000  cats
032
    0.000  cats
033
    1.000  cats
034
    1.000  cats
035
    0.000  cats
036
    1.000  cats
037
    1.000  cats
038
    0.000  cats
039
    1.000  cats
040
    0.000  cats
041
    0.000  cats
042
    1.000  cats
043
    1.000  cats
044
    1.000  cats
045
    0.000  cats
046
    0.999  cats
047
    1.000  cats
048
    0.005  cats
049
    1.000  cats


,ID,Predicted
0,000,0.999992847443
1,001,0.000000215550
2,002,0.999994874001
3,003,0.000000000000
4,004,0.999979734421
5,005,1.000000000000
6,006,0.000000004367
7,007,1.000000000000
8,008,1.000000000000
9,009,0.999887824059


In [24]:
df.head()

,ID,Predicted
0,000,0.999992847443
1,001,0.000000215550
2,002,0.999994874001
3,003,0.000000000000
4,004,0.999979734421


In [26]:
df.to_csv('submission.csv', index=False)